In [6]:

# coding: utf-8

# In[ ]:


from pyspark.sql import SparkSession, SQLContext
import configparser

class ShudCrawler:
    def __init__(self):
        self.config = configparser.ConfigParser()
        self.config.read('../shud.ini')
        
        self.sparkSession = SparkSession\
            .builder\
            .appName(self.config.get('spark', 'appname'))\
            .config("spark.some.config.option", "some-value")\
            .getOrCreate()
        self.sqlContext = SQLContext(self.sparkSession)    
        

        
        
    def start_requests(self):
        """Initialize the processing"""
        initList = [("https://h_amazon.com/DealsHomeUrl", "false")]        
        df = self.sparkSession.createDataFrame(initList, schema=["url", "parsed"])        
        self.sqlContext.registerDataFrameAsTable(df, "WorkTable")

        df = self.sqlContext.sql("SELECT url from WorkTable where parsed = 'false'")
        
        
        indx = 0
        
        while len(df.rdd.collect()) > 0:
            print("Tour ", indx)
            for url in df.rdd.collect():
                self.parse(url[0], indx)                
            
            df = self.sqlContext.sql("SELECT url from WorkTable where parsed = 'false'")
            
            df2 = self.sqlContext.sql("SELECT * from WorkTable")
            df2.show()
            
            if indx > 3:
                break
            indx +=1
            
        
    def parse(self, url, indx):
        # Parse url and add newly found urls
        newUrls = []
        #newUrls.append(("https://x1_amazon.com/x1", "false"))
        if indx < 6:
            newUrls.append(("https://x" + str(indx) + "_amazon.com/x"+ str(indx), "false"))
        
        #Set current url as parsed
        #newUrls.append((url, "true"))
        
        #Get all urls to synchronize and update
        df = self.sqlContext\
            .sql("SELECT url, parsed from WorkTable where url <>'%s'" % url)\
            .union(self.sparkSession.createDataFrame(newUrls))\
            .union(self.sparkSession.createDataFrame([(url, "true")]))\
            .dropDuplicates(['url'])
        
        self.sqlContext.registerDataFrameAsTable(df, "WorkTable")
        

#Testing
st = ShudCrawler()
st.start_requests()


Tour  0
+--------------------+------+
|                 url|parsed|
+--------------------+------+
|https://x0_amazon...| false|
|https://h_amazon....|  true|
+--------------------+------+

Tour  1
+--------------------+------+
|                 url|parsed|
+--------------------+------+
|https://x0_amazon...|  true|
|https://x1_amazon...| false|
|https://h_amazon....|  true|
+--------------------+------+

Tour  2
+--------------------+------+
|                 url|parsed|
+--------------------+------+
|https://x0_amazon...|  true|
|https://x1_amazon...|  true|
|https://h_amazon....|  true|
|https://x2_amazon...| false|
+--------------------+------+

Tour  3
+--------------------+------+
|                 url|parsed|
+--------------------+------+
|https://x0_amazon...|  true|
|https://x3_amazon...| false|
|https://x1_amazon...|  true|
|https://h_amazon....|  true|
|https://x2_amazon...|  true|
+--------------------+------+

Tour  4
+--------------------+------+
|                 url|pars